In [106]:
##################### Import Libraries ############################
import pybamm
import numpy as np
import matplotlib.pyplot as plt

In [107]:
#################### Initiate Blank Model #########################
model = pybamm.BaseModel()
model.name = "Marinescu_2018"

In [108]:
################### Variables #####################################
S8 = pybamm.Variable("S8")
S4 = pybamm.Variable("S4")
S2 = pybamm.Variable("S2")
S  = pybamm.Variable("S")
Sp = pybamm.Variable("Sp")
Ss = pybamm.Variable("Ss")
V  = pybamm.Variable("V")

In [109]:
# Model Variables
model.variables = {"S8" : S8,"S4" : S4,"S2" : S2,"S" : S,"Sp" : Sp,"Ss" : Ss,"V" : V}

In [110]:
##################### Parameters #################################
R = 8.3145
T = 298
F = 9.649*(10**4)
v = 0.0114

EH0 = 2.35
EL0 = 2.195

k_p    = 100
k_s    = 0.0002
f_s    = 0.25
S_star = 0.0001
rho_s  = 2*(10**3)

Ms8 = 32
ne  = 4
ns  = 1
ns2 = 2
ns4 = 4
ns8 = 8
n4  = 4

ih0 = 10
il0 = 5
ar  = 0.960
m_s = 2.7
I   = 1.7

i_h_term_coef   = ns8*Ms8*I*v*rho_s/(ne*F*k_p*(m_s**2))

i_coef   = ne*F/(2*R*T)
i_h_coef = -2*ih0*ar
i_l_coef = -2*il0*ar

E_H_coef = R*T/(4*F)
f_h      = (ns4**2)*Ms8*v/ns8
f_l      = (ns**2)*ns2*Ms8**2*(v**2)/ns4

In [111]:
################# Nested Functions With PyBaMM ##################################

# Nernst Potentials

def E_H(S8,S4,EH0,E_H_coef,f_h):
    return EH0 + E_H_coef*pybamm.log(f_h*S8/((S4**2)))

def E_L(S,S2,S4,EL0,E_H_coef,f_l):
    return EL0 + E_H_coef*pybamm.log(f_l*S4/(S2*(S**2)))

# Surface Overpotentials

def eta_H(V,S8,S4,EH0,E_H_coef,f_h):
    return V-E_H(S8,S4,EH0,E_H_coef,f_h)

def eta_L(V,S,S2,S4,EL0,E_H_coef,f_l):
    return V-E_L(S,S2,S4,EL0,E_H_coef,f_l,)

# Half-cell Currents

def i_H(V,S8,S4,EH0,E_H_coef,f_h,i_coef,i_h_coef):
    return i_h_coef*pybamm.sinh(i_coef*eta_H(V,S8,S4,EH0,E_H_coef,f_h))

def i_L(V,S,S2,S4,EL0,E_H_coef,f_l,i_coef,i_l_coef):
    return i_l_coef*pybamm.sinh(i_coef*eta_L(V,S,S2,S4,EL0,E_H_coef,f_l))

In [112]:
################### Dynamic Equations ########################################
# ODEs
dS8dt = -(ns8*Ms8*i_H(V,S8,S4,EH0,E_H_coef,f_h,i_coef,i_h_coef)/n4*F) - k_s*S8
dS4dt = (2*ns4*Ms8*i_H(V,S8,S4,EH0,E_H_coef,f_h,i_coef,i_h_coef)/n4*F) + (1-(f_s*Ss/m_s))*k_s*S8 - (ns4*Ms8*i_L(V,S,S2,S4,EL0,E_H_coef,f_l,i_coef,i_l_coef)/n4*F)
dS2dt = ns2*Ms8*i_L(V,S,S2,S4,EL0,E_H_coef,f_l,i_coef,i_l_coef)/n4*F
dSdt  = (2*ns*Ms8*i_L(V,S,S2,S4,EL0,E_H_coef,f_l,i_coef,i_l_coef)/n4*F) - (k_p*Sp*(S-S_star)/v*rho_s)
dSpdt = k_p*Sp*(S-S_star)/v*rho_s
dSsdt = k_s*S8

# Algebraic Condition
algebraic_condition = I - i_H(V,S8,S4,EH0,E_H_coef,f_h,i_coef,i_h_coef) - i_L(V,S,S2,S4,EL0,E_H_coef,f_l,i_coef,i_l_coef)

In [113]:
############# Model implementation ###############################################
model.rhs = { S8 : dS8dt, S4 : dS4dt, S2 : dS2dt, S  : dSdt, Sp : dSpdt, Ss : dSsdt }
model.algebraic = {V : algebraic_condition}

In [114]:
########################## Given Initial Conditions ####################################
# Given values (dimensional)
S8_initial = 0.998*m_s
S4_initial = 0.001*m_s
S_initial  = S_star*m_s
Ss_initial = 0
I_initial  = 0

In [115]:
################# Nested Functions With Numpy ##################################

# Nernst Potentials

def E_H_np(S8,S4,EH0,E_H_coef,f_h):
    return EH0 + E_H_coef*np.log(f_h*S8/((S4**2)))

def E_L_np(S,S2,S4,EL0,E_H_coef,f_l):
    return EL0 + E_H_coef*np.log(f_l*S4/(S2*(S**2)))


In [116]:
########################## Derived Initial Conditions ##################################

import scipy as sp

V_initial = E_H_np(S8_initial,S4_initial,EH0,E_H_coef,f_h)

S2_initial = np.exp(n4*F*(EH0-V_initial)/(R*T))*(S_initial/(f_l*S4_initial))

Sp_initial = m_s - S8_initial - S4_initial - S2_initial - S_initial - Ss_initial

In [126]:
# Initial Conditions
model.initial_conditions = {S8 : pybamm.Scalar(S8_initial), S4 : pybamm.Scalar(S4_initial),S2 : pybamm.Scalar(S2_initial),S : pybamm.Scalar(S_initial),Sp : pybamm.Scalar(Sp_initial),Ss : pybamm.Scalar(Ss_initial), V : pybamm.Scalar(V_initial)}

Now I am going to implement the model in almost the exact same way as shown in the documentation.

In [128]:
disc = pybamm.Discretisation()  # use the default discretisation
disc.process_model(model);

In [139]:
#import scipy.Scikits.ode
solver = pybamm.ScipySolver()#ScikitsDaeSolver() #This I changed since ScipySolver doesn't handle DAE's
t = np.linspace(0, 1, 20)
solution = solver.solve(model, t)

SolverError: Cannot use ODE solver 'Scipy solver (BDF)' to solve DAE model

In [138]:
pybamm.

ImportError: cannot import name 'SciKits' from 'scipy' (/Users/michaelcornish/opt/anaconda3/lib/python3.7/site-packages/scipy/__init__.py)